# Proyecto Final IH: Recomendador de Codewars|Github

In [ ]:
# https://www.codewars.com/users/leaderboard

In [2]:
# imports 

import pandas as pd
import numpy as np
import requests
import json
from bs4 import BeautifulSoup
from IPython.display import display

pd.options.display.max_columns = None

## Fase 1: Obtención de usuarios de forma iterativa

### Leaderboard
Top 500 de codewars

In [3]:
# URL

url = 'https://www.codewars.com/users/leaderboard'
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")

In [4]:
# usuarios de leaderboard
def get_top_500(url='https://www.codewars.com/users/leaderboard'): 
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    return {e.text for e in soup.select('tr a')}

# users = '\n'.join(get_top_500())
# set_top500
# users
# get_top_500()

In [5]:
# Salvamos los nombres en un documento
def save_set_users(users, filename='../output/codewars-users.txt'): 
    ''' Save set/list of users to file '''
    with open(filename, 'w') as f: 
        f.write('\n'.join(users)) 
# save_set_users({'hola', 'adios'})

In [6]:
def save_string_users(users, filename='../output/codewars-users.txt'): 
    ''' Save string of users to file '''
    with open(filename, 'w') as f: 
        f.write(users) 
# save_set_users(users)

In [7]:
def add_set_users(users, filename='../output/codewars-users.txt'): 
    ''' Append to file '''
    with open(filename, 'a') as f: 
        f.write(users) 
# add_set_users(users)

In [8]:
def load_set_users(filename='../output/codewars-users.txt'): 
    ''' Read from file '''
    with open(filename, 'r') as f: 
        users = f.readlines()
    return {user.strip() for user in users}
# len(load_set_users())

### Get Social

In [9]:
# get users in clan

# https://www.codewars.com/users/albertogcmr/following
# https://www.codewars.com/users/albertogcmr/followers
# https://www.codewars.com/users/albertogcmr/allies

def get_social(user): 
    res = set()
    for link in ['following', 'followers', 'allies']: 
        url = 'https://www.codewars.com/users/{}/{}'.format(user, link)
        html = requests.get(url).content

        soup = BeautifulSoup(html, "lxml")
        res = res.union({e.text for e in soup.select('table a')})
    return res

# get_social('albertogcmr')

In [11]:
### Get stats from user
'''
def get_all_stats(user): 
    url = 'https://www.codewars.com/users/{}'.format(user)
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")

    x = {}
    x['username'] = user
    for s in soup.select('.stat-box div'): 
        if s.text.split(':')[0] != 'Profiles': 
            x[s.text.split(':')[0].lower()] = s.text.split(':')[1]
        else: 
            try: 
                for e in s.find_all('a', href=True): 
                    if 'github' in e['href']: 
                        x['github'] = e['href']
                    if 'linkedin' in e['href']: 
                        x['linkedin'] = e['href']
            except: 
                x['github'] = ''
                x['linkedin'] = ''
    return x

get_all_stats('albertogcmr')

'''
pass

### Iteración para ampliar número de usuarios

A partir de la semilla inicial de 500 obtenemos casi 4000 usuarios de codewars

In [ ]:
top500 = get_top_500()
total = set().union(top500)
users = total

In [ ]:
print(len(total))

for i, user in enumerate(users): 
    total = total.union(get_social(user))
    print(i, '\t', len(total))

print(len(total))

In [ ]:
# Me añado a mí mismo

total = total.union({'albertogcmr'})
for user in get_social('albertogcmr'): 
    total = total.union(get_social(user))
len(total)

### Persistencia de datos en un txt

In [18]:
# save_set_users(users=total, filename='../output/no-borrar-usuarioscodewars-2.txt')
total = load_set_users('../output/no-borrar-usuarioscodewars-2.txt')

In [19]:
len(total)

3953

## Fase 2: Obtención de datos de usuario
### 2.1 API codewars

In [20]:
# GET user

def get_user_api(user='albertogcmr'): 
    ''' Obtenemos el json de la API de codewars '''
    url = 'https://www.codewars.com/api/v1/users/{}'.format(user)
    response = requests.get(url)
    return response.json()

get_user_api('albertogcmr')

{'username': 'albertogcmr',
 'name': 'Alberto García cobo',
 'honor': 727,
 'clan': 'Ironhack',
 'leaderboardPosition': 12892,
 'skills': ['java',
  'c',
  'python',
  'jquery',
  'javascript',
  'crm',
  'erp',
  'itil'],
 'ranks': {'overall': {'rank': -4,
   'name': '4 kyu',
   'color': 'blue',
   'score': 888},
  'languages': {'python': {'rank': -4,
    'name': '4 kyu',
    'color': 'blue',
    'score': 857},
   'sql': {'rank': -7, 'name': '7 kyu', 'color': 'white', 'score': 24},
   'c': {'rank': -8, 'name': '8 kyu', 'color': 'white', 'score': 10}}},
 'codeChallenges': {'totalAuthored': 0, 'totalCompleted': 89}}

In [21]:
def get_value_from_json(user_json, value='username'): 
    ''' 
    value en nivel 1 del json: 
    possible values = ['username', 'honor', 'clan', 'leaderboardPosition', 'skills']
    '''
    return user_json[value]

def get_score_language(user_json, language): 
    try: 
        score = user_json['ranks']['languages'][language]['score']
    except: 
        score = 0
    return score


def get_scores(user_json): 
    res = {}
    for lang in get_languages(): 
        res[lang] = get_score_language(user_json, lang)
    return res

# get_score_language(user_json=alberto, language='sca')
# get_value_from_json(alberto, 'skills')
# get_scores(alberto)

### 2.2 Web Scrapping codewars

Get stats from user

In [23]:
def get_all_stats(user): 
    url = 'https://www.codewars.com/users/{}'.format(user)
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")

    x = {}
    x['username'] = user
    for s in soup.select('.stat-box div'): 
        if s.text.split(':')[0] != 'Profiles': 
            x[s.text.split(':')[0].lower()] = s.text.split(':')[1]
        else: 
            try: 
                for e in s.find_all('a', href=True): 
                    if 'github' in e['href']: 
                        x['github'] = e['href']
                    if 'linkedin' in e['href']: 
                        x['linkedin'] = e['href']
            except: 
                x['github'] = ''
                x['linkedin'] = ''
    return x

get_all_stats('albertogcmr')

{'username': 'albertogcmr',
 'name': 'Alberto García cobo',
 'clan': 'Ironhack',
 'skills': 'java, c, python, jquery, javascript, crm, erp, itil',
 'member since': 'Oct 2018',
 'last seen': 'Dec 2018',
 'github': 'https://github.com/albertogcmr',
 'linkedin': 'https://www.linkedin.com/in/albertogarciacobo/',
 'following': '593',
 'followers': '591',
 'allies': '591',
 'rank': '4 kyu',
 'honor': '727',
 'leaderboard position': '#12,892',
 'honor percentile': '98th',
 'total completed kata': '89',
 'total languages trained': '3',
 'highest trained': 'Python (4 kyu)',
 'most recent': 'Python',
 'comments': '0 (0 replies)',
 'collections': '0',
 'kumite': '0',
 'translations': '0 (0 approved)'}

In [25]:
def get_languages(url='https://www.codewars.com/kata/latest/my-languages', ignore={'all', 'my languages'}): 
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    return {e.text.lower() for e in soup.select('#language_filter option')}.difference(ignore) 

get_languages()

{'bf (beta)',
 'c',
 'c#',
 'c++',
 'clojure',
 'coffeescript',
 'crystal',
 'dart',
 'elixir',
 'elm (beta)',
 'erlang (beta)',
 'f#',
 'fortran (beta)',
 'go',
 'groovy (beta)',
 'haskell',
 'java',
 'javascript',
 'julia (beta)',
 'kotlin (beta)',
 'lua (beta)',
 'nasm (beta)',
 'nim (beta)',
 'objective-c (beta)',
 'ocaml (beta)',
 'php',
 'powershell (beta)',
 'purescript (beta)',
 'python',
 'r (beta)',
 'ruby',
 'rust',
 'scala (beta)',
 'shell',
 'solidity (beta)',
 'sql',
 'swift',
 'typescript'}

In [26]:
def get_row(user): 
    ''' 
    Crea un diccionario con los datos recopilados que se puede
    añadir como fila en nuestro dataframe de pandas
    '''
    user_json = get_user_api(user)
    res = get_all_stats(user)
    res.update(get_scores(user_json))
    return res

# ivan = get_row('ijcernicharo')
# alberto = get_row('albertogcmr')

In [29]:
def create_row(data, new_user): 
    row = get_row(new_user)
    return data.append(other=row, ignore_index=True)

### 2.3 Creación de DataFrame

In [57]:
df_users = pd.DataFrame()

In [58]:
# Por cada uno de los usuarios recopilados, creamos un registro en df_users
total_10 = [u for u in total][:3]
for i, user in enumerate(total_10): 
    df_users = create_row(data=df_users, new_user=user)
    print(i)
df_users.head()

0
1
2


,allies,bf (beta),c,c#,c++,clan,clojure,coffeescript,collections,comments,crystal,dart,elixir,elm (beta),erlang (beta),f#,followers,following,fortran (beta),github,go,groovy (beta),haskell,highest trained,honor,honor percentile,java,javascript,julia (beta),kotlin (beta),kumite,last seen,leaderboard position,lua (beta),member since,most recent,name,nasm (beta),nim (beta),objective-c (beta),ocaml (beta),php,powershell (beta),purescript (beta),python,r (beta),rank,ruby,rust,scala (beta),shell,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,username
0,1,0.0,0.0,0.0,0.0,Unknown,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,https://github.com/dynelight,0.0,0.0,0.0,Ruby (6 kyu),135,83rd,0.0,0.0,0.0,0.0,0,Jul 2018,"#110,151",0.0,Jul 2018,Ruby,Esteban Rodriguez,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6 kyu,118.0,0.0,0.0,0.0,0.0,0.0,0.0,20,1,0 (0 approved),0.0,dynelight
1,1,0.0,0.0,0.0,0.0,Unknown,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,https://github.com/trahkoi,0.0,0.0,24.0,Python (6 kyu),155,85th,0.0,4.0,0.0,0.0,0,Jul 2016,"#93,348",0.0,Sep 2014,Python,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.0,0.0,6 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28,3,0 (0 approved),0.0,trahkoi
2,10,0.0,0.0,0.0,0.0,Frosmo Ltd,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,39,44,0.0,NaN,0.0,0.0,0.0,JavaScript (4 kyu),562,96th,0.0,1040.0,0.0,0.0,0,Jul 2017,"#19,767",0.0,Aug 2015,JavaScript,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4 kyu,3.0,0.0,0.0,0.0,0.0,0.0,0.0,48,2,0 (0 approved),0.0,Sensei_Frosmo


In [59]:
df_users = create_row(data=df_users, new_user='ijcernicharo')
df_users = create_row(data=df_users, new_user='pablobarrio')
df_users.head()

,allies,bf (beta),c,c#,c++,clan,clojure,coffeescript,collections,comments,crystal,dart,elixir,elm (beta),erlang (beta),f#,followers,following,fortran (beta),github,go,groovy (beta),haskell,highest trained,honor,honor percentile,java,javascript,julia (beta),kotlin (beta),kumite,last seen,leaderboard position,lua (beta),member since,most recent,name,nasm (beta),nim (beta),objective-c (beta),ocaml (beta),php,powershell (beta),purescript (beta),python,r (beta),rank,ruby,rust,scala (beta),shell,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,username,skills
0,1,0.0,0.0,0.0,0.0,Unknown,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,https://github.com/dynelight,0.0,0.0,0.0,Ruby (6 kyu),135,83rd,0.0,0.0,0.0,0.0,0,Jul 2018,"#110,151",0.0,Jul 2018,Ruby,Esteban Rodriguez,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6 kyu,118.0,0.0,0.0,0.0,0.0,0.0,0.0,20,1,0 (0 approved),0.0,dynelight,NaN
1,1,0.0,0.0,0.0,0.0,Unknown,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,https://github.com/trahkoi,0.0,0.0,24.0,Python (6 kyu),155,85th,0.0,4.0,0.0,0.0,0,Jul 2016,"#93,348",0.0,Sep 2014,Python,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.0,0.0,6 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28,3,0 (0 approved),0.0,trahkoi,NaN
2,10,0.0,0.0,0.0,0.0,Frosmo Ltd,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,39,44,0.0,NaN,0.0,0.0,0.0,JavaScript (4 kyu),562,96th,0.0,1040.0,0.0,0.0,0,Jul 2017,"#19,767",0.0,Aug 2015,JavaScript,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4 kyu,3.0,0.0,0.0,0.0,0.0,0.0,0.0,48,2,0 (0 approved),0.0,Sensei_Frosmo,NaN
3,101,0.0,0.0,0.0,0.0,IronHack,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,101,102,0.0,https://github.com/ijcernicharo,0.0,0.0,0.0,Python (5 kyu),423,95th,0.0,0.0,0.0,0.0,0,Dec 2018,"#28,059",0.0,Oct 2018,Python,Ivan Cernicharo Ortiz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,316.0,0.0,5 kyu,0.0,0.0,0.0,0.0,0.0,47.0,0.0,53,2,0 (0 approved),0.0,ijcernicharo,"isaac asimov nerd, python, matlab, sql, qgis (..."
4,0,0.0,0.0,0.0,0.0,Unknown,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,https://github.com/pablobarrio,0.0,0.0,0.0,Python (6 kyu),134,82nd,0.0,0.0,0.0,0.0,0,Dec 2018,"#110,950",0.0,Oct 2018,Python,pablobarrio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.0,0.0,6 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26,1,0 (0 approved),0.0,pablobarrio,NaN


In [60]:
'linkedin' in df_users.columns

False

In [64]:
df_users = create_row(data=df_users, new_user='albertogcmr')
df_users.head(7)

,allies,bf (beta),c,c#,c++,clan,clojure,coffeescript,collections,comments,crystal,dart,elixir,elm (beta),erlang (beta),f#,followers,following,fortran (beta),github,go,groovy (beta),haskell,highest trained,honor,honor percentile,java,javascript,julia (beta),kotlin (beta),kumite,last seen,leaderboard position,lua (beta),member since,most recent,name,nasm (beta),nim (beta),objective-c (beta),ocaml (beta),php,powershell (beta),purescript (beta),python,r (beta),rank,ruby,rust,scala (beta),shell,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,username,skills,linkedin
0,1,0.0,0.0,0.0,0.0,Unknown,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,https://github.com/dynelight,0.0,0.0,0.0,Ruby (6 kyu),135,83rd,0.0,0.0,0.0,0.0,0,Jul 2018,"#110,151",0.0,Jul 2018,Ruby,Esteban Rodriguez,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6 kyu,118.0,0.0,0.0,0.0,0.0,0.0,0.0,20,1,0 (0 approved),0.0,dynelight,NaN,NaN
1,1,0.0,0.0,0.0,0.0,Unknown,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,https://github.com/trahkoi,0.0,0.0,24.0,Python (6 kyu),155,85th,0.0,4.0,0.0,0.0,0,Jul 2016,"#93,348",0.0,Sep 2014,Python,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.0,0.0,6 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28,3,0 (0 approved),0.0,trahkoi,NaN,NaN
2,10,0.0,0.0,0.0,0.0,Frosmo Ltd,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,39,44,0.0,NaN,0.0,0.0,0.0,JavaScript (4 kyu),562,96th,0.0,1040.0,0.0,0.0,0,Jul 2017,"#19,767",0.0,Aug 2015,JavaScript,Unknown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4 kyu,3.0,0.0,0.0,0.0,0.0,0.0,0.0,48,2,0 (0 approved),0.0,Sensei_Frosmo,NaN,NaN
3,101,0.0,0.0,0.0,0.0,IronHack,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,101,102,0.0,https://github.com/ijcernicharo,0.0,0.0,0.0,Python (5 kyu),423,95th,0.0,0.0,0.0,0.0,0,Dec 2018,"#28,059",0.0,Oct 2018,Python,Ivan Cernicharo Ortiz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,316.0,0.0,5 kyu,0.0,0.0,0.0,0.0,0.0,47.0,0.0,53,2,0 (0 approved),0.0,ijcernicharo,"isaac asimov nerd, python, matlab, sql, qgis (...",NaN
4,0,0.0,0.0,0.0,0.0,Unknown,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,https://github.com/pablobarrio,0.0,0.0,0.0,Python (6 kyu),134,82nd,0.0,0.0,0.0,0.0,0,Dec 2018,"#110,950",0.0,Oct 2018,Python,pablobarrio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.0,0.0,6 kyu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26,1,0 (0 approved),0.0,pablobarrio,NaN,NaN
5,591,0.0,10.0,0.0,0.0,Ironhack,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,591,593,0.0,https://github.com/albertogcmr,0.0,0.0,0.0,Python (4 kyu),727,98th,0.0,0.0,0.0,0.0,0,Dec 2018,"#12,892",0.0,Oct 2018,Python,Alberto García cobo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,857.0,0.0,4 kyu,0.0,0.0,0.0,0.0,0.0,24.0,0.0,89,3,0 (0 approved),0.0,albertogcmr,"java, c, python, jquery, javascript, crm, erp,...",https://www.linkedin.com/in/albertogarciacobo/
6,591,0.0,10.0,0.0,0.0,Ironhack,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,591,593,0.0,https://github.com/albertogcmr,0.0,0.0,0.0,Python (4 kyu),727,98th,0.0,0.0,0.0,0.0,0,Dec 2018,"#12,892",0.0,Oct 2018,Python,Alberto García cobo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,857.0,0.0,4 kyu,0.0,0.0,0.0,0.0,0.0,24.0,0.0,89,3,0 (0 approved),0.0,albertogcmr,"java, c, python, jquery, javascript, crm, erp,...",https://www.linkedin.com/in/albertogarciacobo/


In [ ]:
'linkedin' in df_users.columns

In [50]:
df_users.to_csv('../output/dataframe.csv')
df = pd.read_csv('../output/dataframe.csv')
df.head()

,Unnamed: 0,allies,bf (beta),c,c#,c++,clan,clojure,coffeescript,collections,comments,crystal,dart,elixir,elm (beta),erlang (beta),f#,followers,following,fortran (beta),github,go,groovy (beta),haskell,highest trained,honor,honor percentile,java,javascript,julia (beta),kotlin (beta),kumite,last seen,leaderboard position,lua (beta),member since,most recent,name,nasm (beta),nim (beta),objective-c (beta),ocaml (beta),php,powershell (beta),purescript (beta),python,r (beta),rank,ruby,rust,scala (beta),shell,skills,solidity (beta),sql,swift,total completed kata,total languages trained,translations,typescript,username,linkedin
0,0,101,0.0,0.0,0.0,0.0,IronHack,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,101,102,0.0,https://github.com/ijcernicharo,0.0,0.0,0.0,Python (5 kyu),423,95th,0.0,0.0,0.0,0.0,0,Dec 2018,"#28,059",0.0,Oct 2018,Python,Ivan Cernicharo Ortiz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,316.0,0.0,5 kyu,0.0,0.0,0.0,0.0,"isaac asimov nerd, python, matlab, sql, qgis (...",0.0,47.0,0.0,53,2,0 (0 approved),0.0,ijcernicharo,NaN
1,1,0,0.0,0.0,0.0,0.0,Unknown,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,https://github.com/pablobarrio,0.0,0.0,0.0,Python (6 kyu),134,82nd,0.0,0.0,0.0,0.0,0,Dec 2018,"#110,950",0.0,Oct 2018,Python,pablobarrio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114.0,0.0,6 kyu,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,26,1,0 (0 approved),0.0,pablobarrio,NaN
2,2,591,0.0,10.0,0.0,0.0,Ironhack,0.0,0.0,0,0 (0 replies),0.0,0.0,0.0,0.0,0.0,0.0,591,593,0.0,https://github.com/albertogcmr,0.0,0.0,0.0,Python (4 kyu),727,98th,0.0,0.0,0.0,0.0,0,Dec 2018,"#12,892",0.0,Oct 2018,Python,Alberto García cobo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,857.0,0.0,4 kyu,0.0,0.0,0.0,0.0,"java, c, python, jquery, javascript, crm, erp,...",0.0,24.0,0.0,89,3,0 (0 approved),0.0,albertogcmr,https://www.linkedin.com/in/albertogarciacobo/


### 2.4 Limpieza de datos

## Fase 3: Persistencia de datos

In [ ]:
Guarda